### Previsão de Mortes Neonatal

Neste projeto, vamos utilizar dados públicos do SINASC (Sistema de Nascidos Vivos) e do SIM (Sistema de Mortalidade) para construir modelos de aprendizado de máquina capazes de prever o risco de morte neonatal (até 28 dias após o nascimento).

*Alunos:*
- Júlia Moraes
- Luiz Eduardo

#### Imports necessários para o projeto

In [23]:
import pandas as pd
import numpy as np
import recordlinkage

#### 1.Datasets

##### 1.1 SINAC - Sistema de Nascidos Vivos 

Iniciaremos fazendo o tratamento e limpeza nos dados do SINAC, que adquirimos no site do Ministério da Saúde.

O Governo disponibiliza o [Dicionário de Dados](https://diaad.s3.sa-east-1.amazonaws.com/sinasc/SINASC+-+Estrutura.pdf) referente ao arquivo .csv utilizado. No entanto, neste trabalho serão abordados apenas os campos relevantes para o desenvolvimento do projeto.


##### Variáveis úteis para antes do parto

- Idade da Mãe (IDADEMAE): extremos de idade tais como adolescentes e mulheres mais velhas que 40 anos apresentam mais risco por fatores biológicos
- Número de consultas pré-natais (CONSULTAS): menos consultas significa mais risco de problemas não detectados
- Início pré-natal (MESPRENAT): início tardio pode significar mais risco de problemas não detectados
- Paridade (PARIDADE): primíparas e multíparas com histórico de perdas podem ter riscos diferentes
- Gravidez múltipla (GRAVIDEZ): gêmeos/trigêmeos têm maior risco de prematuridade e baixo peso.

##### Variáveis no nascimento
- Peso ao nascer (PESO): peso baixo ao nascer são fortes preditores de mortalidade
- Idade Gestacional (SEMAGESTAC): principal determinante
- Apgar 1 e 5 minutos (APGAR1, APGAR5): medida imediata da condição neonatal, baixo Apgar indica sofrimento perinatal e forte preditor nos primeiros dias
- Anomalias congênitas (IDANOMAL): se possuir pode aumentar o risco de morte neonatal
- Tipo de parto (PARTO): parto de emergência muitas vezes reflete complicação fetal/materna
- Índice de Kotelchuck ( KOTELCHUCK): avaliação da assistência pré-natal

In [4]:
sinac = pd.read_csv(r"G:\Meu Drive\datasets\SINASC_2023.csv", sep=";")

C:\Users\julja\AppData\Local\Temp\ipykernel_8724\783069641.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  sinac = pd.read_csv(r"G:\Meu Drive\datasets\SINASC_2023.csv", sep=";")


In [5]:
# ver as primeiras 5 linhas 
sinac.head()

,contador,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,...,TPFUNCRESP,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK
0,1,1,2679477.0,110001,1,32.0,2.0,4.0,999992.0,3.0,...,2.0,3.0,2012023.0,6.0,0.0,1,1.0,3,1,5
1,2,1,2679477.0,110001,1,18.0,1.0,4.0,999992.0,NaN,...,2.0,3.0,25012023.0,6.0,0.0,1,1.0,10,0,5
2,3,1,2679477.0,110001,1,15.0,5.0,3.0,999992.0,0.0,...,2.0,3.0,10022023.0,3.0,0.0,1,1.0,6,0,3
3,4,1,2516500.0,110001,1,32.0,2.0,3.0,999992.0,1.0,...,2.0,3.0,18012023.0,3.0,0.0,1,1.0,5,1,5
4,5,1,2516500.0,110001,1,27.0,1.0,5.0,999991.0,0.0,...,2.0,3.0,18012023.0,8.0,0.0,1,1.0,1,0,5


In [6]:
# quantidade de linhas e colunas
sinac.shape

(2537576, 62)

In [7]:
# informações gerais
sinac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2537576 entries, 0 to 2537575
Data columns (total 62 columns):
 #   Column      Dtype  
---  ------      -----  
 0   contador    int64  
 1   ORIGEM      int64  
 2   CODESTAB    float64
 3   CODMUNNASC  int64  
 4   LOCNASC     int64  
 5   IDADEMAE    float64
 6   ESTCIVMAE   float64
 7   ESCMAE      float64
 8   CODOCUPMAE  float64
 9   QTDFILVIVO  float64
 10  QTDFILMORT  float64
 11  CODMUNRES   int64  
 12  GESTACAO    float64
 13  GRAVIDEZ    float64
 14  PARTO       float64
 15  CONSULTAS   float64
 16  DTNASC      int64  
 17  HORANASC    float64
 18  SEXO        int64  
 19  APGAR1      float64
 20  APGAR5      float64
 21  RACACOR     float64
 22  PESO        float64
 23  IDANOMAL    float64
 24  DTCADASTRO  int64  
 25  CODANOMAL   object 
 26  NUMEROLOTE  float64
 27  VERSAOSIST  object 
 28  DTRECEBIM   float64
 29  DIFDATA     int64  
 30  OPORT_DN    int64  
 31  DTRECORIGA  int64  
 32  NATURALMAE  float64
 33  CODMUNN

In [8]:
# estatísticas descritivas
sinac.describe(include="all")

,contador,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,...,TPFUNCRESP,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK
count,2.537576e+06,2.537576e+06,2.512801e+06,2.537576e+06,2.537576e+06,2.537553e+06,2.527680e+06,2.527838e+06,2.356750e+06,2.500450e+06,...,2.474101e+06,2.527016e+06,2.513036e+06,2.517499e+06,2.537573e+06,2.537576e+06,2537555.0,2.537576e+06,2.537576e+06,2.537576e+06
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.268788e+06,1.000139e+00,2.994136e+06,3.202480e+05,1.030377e+00,2.765736e+01,2.006280e+00,4.122294e+00,6.898684e+05,1.027298e+00,...,2.856320e+00,3.220967e+00,1.578357e+07,6.347344e+00,6.774816e-01,9.999882e-01,1.0,4.155663e+00,6.351045e-01,4.473895e+00
std,7.325352e+05,1.179364e-02,1.999851e+06,1.006997e+05,2.624615e-01,6.724978e+00,1.451708e+00,6.814163e-01,3.058799e+05,1.352322e+00,...,1.424529e+00,1.033406e+00,8.776789e+06,2.339970e+00,4.674402e-01,3.438338e-03,0.0,2.698849e+00,4.814009e-01,1.398146e+00
min,1.000000e+00,1.000000e+00,9.400000e+01,1.100010e+05,1.000000e+00,8.000000e+00,1.000000e+00,1.000000e+00,1.020500e+04,0.000000e+00,...,0.000000e+00,0.000000e+00,1.012023e+06,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.000000e+00,0.000000e+00,1.000000e+00
25%,6.343948e+05,1.000000e+00,2.119528e+06,2.601200e+05,1.000000e+00,2.200000e+01,1.000000e+00,4.000000e+00,4.132050e+05,0.000000e+00,...,2.000000e+00,3.000000e+00,8.082023e+06,5.000000e+00,0.000000e+00,1.000000e+00,1.0,2.000000e+00,0.000000e+00,4.000000e+00
50%,1.268788e+06,1.000000e+00,2.461005e+06,3.205300e+05,1.000000e+00,2.700000e+01,1.000000e+00,4.000000e+00,6.220200e+05,1.000000e+00,...,2.000000e+00,3.000000e+00,1.602202e+07,6.000000e+00,1.000000e+00,1.000000e+00,1.0,4.000000e+00,1.000000e+00,5.000000e+00
75%,1.903182e+06,1.000000e+00,2.798220e+06,3.550300e+05,1.000000e+00,3.300000e+01,2.000000e+00,4.000000e+00,9.999920e+05,2.000000e+00,...,5.000000e+00,4.000000e+00,2.310202e+07,8.000000e+00,1.000000e+00,1.000000e+00,1.0,5.000000e+00,1.000000e+00,5.000000e+00


In [9]:
# verificando porcentagem de valores nulos
sinac.isnull().mean() * 100

contador      0.000000
ORIGEM        0.000000
CODESTAB      0.976325
CODMUNNASC    0.000000
LOCNASC       0.000000
                ...   
STDNNOVA      0.000000
CODPAISRES    0.000828
TPROBSON      0.000000
PARIDADE      0.000000
KOTELCHUCK    0.000000
Length: 62, dtype: float64

In [10]:
# verificando os tipos de dados de cada coluna
for coluna, tipo in sinac.dtypes.items():
    print(f"{coluna}: {tipo}")

contador: int64
ORIGEM: int64
CODESTAB: float64
CODMUNNASC: int64
LOCNASC: int64
IDADEMAE: float64
ESTCIVMAE: float64
ESCMAE: float64
CODOCUPMAE: float64
QTDFILVIVO: float64
QTDFILMORT: float64
CODMUNRES: int64
GESTACAO: float64
GRAVIDEZ: float64
PARTO: float64
CONSULTAS: float64
DTNASC: int64
HORANASC: float64
SEXO: int64
APGAR1: float64
APGAR5: float64
RACACOR: float64
PESO: float64
IDANOMAL: float64
DTCADASTRO: int64
CODANOMAL: object
NUMEROLOTE: float64
VERSAOSIST: object
DTRECEBIM: float64
DIFDATA: int64
OPORT_DN: int64
DTRECORIGA: int64
NATURALMAE: float64
CODMUNNATU: float64
CODUFNATU: object
ESCMAE2010: float64
SERIESCMAE: float64
DTNASCMAE: float64
RACACORMAE: float64
QTDGESTANT: float64
QTDPARTNOR: float64
QTDPARTCES: float64
IDADEPAI: float64
DTULTMENST: float64
SEMAGESTAC: float64
TPMETESTIM: float64
CONSPRENAT: float64
MESPRENAT: float64
TPAPRESENT: float64
STTRABPART: float64
STCESPARTO: float64
TPNASCASSI: float64
TPFUNCRESP: float64
TPDOCRESP: float64
DTDECLARAC: float6

In [11]:
# renomear colunas para facilitar o entendimento
sinac.rename(columns={
    "IDADEMAE": "idade_mae",
    "CONSULTAS": "consultas_prenatal",
    "MESPRENAT": "mes_inicio_prenatal",
    "PARIDADE": "paridade",
    "GRAVIDEZ": "tipo_gravidez",
    "PESO": "peso_bebe",
    "SEMAGESTAC": "semanas_gestacao",
    "APGAR1": "apgar_1min",
    "APGAR5": "apgar_5min",
    "IDANOMAL": "anomalia_identificada",
    "PARTO": "tipo_parto",
    "KOTELCHUCK": "indice_kotelchuck",
    "ESCMAE": "escolaridade_mae"
}, inplace=True)

In [13]:
# remover linhas duplicadas
sinac.drop_duplicates(inplace=True)

Criação de variáveis clinicamente relevantes para a predição 

In [14]:
# Baixo peso e muito baixo peso
sinac["baixo_peso"] = (sinac["peso_bebe"] < 2500).astype(int)
sinac["muito_baixo_peso"] = (sinac["peso_bebe"] < 1500).astype(int)

In [15]:
# Prematuro ou prematuro extremo
sinac["prematuro"] = (sinac["semanas_gestacao"] < 37).astype(int)
sinac["prematuro_extremo"] = (sinac["semanas_gestacao"] < 28).astype(int)

In [16]:
# Apgar crítico ou baixo
sinac['apgar5_baixo'] = (sinac['apgar_5min'] < 7).astype(int)
sinac['apgar1_critico'] = (sinac['apgar_1min'] < 4).astype(int)

# Apgar sem melhoria entre o 1 e 5 minutos
sinac['apgar_sem_melhoria'] = ((sinac['apgar_5min'] - sinac['apgar_1min']) <= 2).astype(int)

In [17]:
#Idades da mãe
sinac['mae_adolescente'] = (sinac['idade_mae'] < 18).astype(int)
sinac['mae_idade_avancada'] = (sinac['idade_mae'] > 35).astype(int)

In [18]:
#Pré natal inadequado
sinac['prenatal_inadequado'] = sinac['indice_kotelchuck'].isin([1, 2]).astype(int)
sinac['poucas_consultas'] = (sinac['consultas_prenatal'] < 4).astype(int)
sinac['prenatal_tardio'] = (sinac['mes_inicio_prenatal'] > 3).astype(int)

In [19]:
# Variáveis compostas
sinac['risco_extremo'] = ((sinac['prematuro_extremo'] == 1) & (sinac['muito_baixo_peso'] == 1)).astype(int)
    
sinac['escore_risco_neonatal'] = (
    (sinac['baixo_peso'] == 1) +
        (sinac['prematuro'] == 1) + 
        (sinac['apgar5_baixo'] == 1) +
        (sinac['anomalia_identificada'] == 1)
    )

sinac['risco_materno'] = (
    (sinac['mae_adolescente'] == 1) +
    (sinac['mae_idade_avancada'] == 1) +
    (sinac['prenatal_inadequado'] == 1)
    )

#### 1.2 SIM (Sistema de mortalidade)

In [39]:
sim = pd.read_csv(r"G:\Meu Drive\datasets\DO23OPEN.csv", sep=";")

C:\Users\julja\AppData\Local\Temp\ipykernel_8724\2605438413.py:1: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  sim = pd.read_csv(r"G:\Meu Drive\datasets\DO23OPEN.csv", sep=";")


In [ ]:
sinac.rename(columns={
    'DTNASC': 'data_nasc',
    'SEXO': 'sexo',
    'CODMUNNASC': 'codmun_res',
    'DTCADASTRO': 'dt_cadastro'
}, inplace=True)

sim.rename(columns={
    'DTOBITO': 'data_obito', 
    'SEXO': 'sexo',
    'CODMUNRES': 'codmun_res',
    'IDADEMAE': 'idade_mae',
    'DTCADASTRO': 'dt_cadastro'
}, inplace=True)
